In [1]:
import numpy as np
import pandas as pd
%matplotlib inline 
import re
from datetime import datetime
from datetime import timedelta

pd.options.display.max_columns = 100

In [2]:
df = pd.read_csv('report-1524833684.csv')

In [3]:
df.head()

,id,sex,ref,age,country,rate.p.0.0,rate.p.0.1,rate.p.1.0,rate.p.1.1,rate.c.0.0,rate.c.0.1,rate.c.0.2,rate.c.0.3,rate.c.1.0,rate.c.1.2
0,8547552,0,NaN,28,Россия,56.25,52.43,30.0,41.67,32.0,18.0,309.0,162.0,18.0,25.0
1,31396167,1,NaN,26,Латвия,26.67,0.00,35.0,0.00,135.0,36.0,1.0,NaN,21.0,0.0
2,118185159,1,1.0,38,Турция,100.00,0.00,5.0,0.00,3.0,3.0,NaN,NaN,3.0,NaN
3,8908784,1,NaN,36,Россия,0.00,13.79,0.0,6.67,9.0,NaN,29.0,4.0,0.0,4.0
4,105444829,1,NaN,20,Украина,0.00,54.55,0.0,10.00,NaN,NaN,11.0,6.0,NaN,6.0


Параметры : 
rate.p.X.Y
p - проценты 
X - типы (0 все время, 1 последние 60)
Y - (0 исходящие лайки, 1 входящие лафки)

rate.с.X.Y
c - счетчики
X - типы (0 все время, 1 последние 60)
Y -
0 - всего исходящих
1 - из них лайков
2 - всего входящих
3 - из них лайков

для типа 1
0 - количество лайков из последних 60 исходящих
2 - количество лайков из последних 60 входящих

In [4]:
df.describe()

,id,sex,ref,age,rate.p.0.0,rate.p.0.1,rate.p.1.0,rate.p.1.1,rate.c.0.0,rate.c.0.1,rate.c.0.2,rate.c.0.3,rate.c.1.0,rate.c.1.2
count,4.331460e+05,433146.000000,85498.0,433146.000000,418480.000000,418480.000000,418480.000000,418480.000000,345263.000000,308363.000000,403092.000000,338932.000000,345263.000000,403092.000000
mean,8.571775e+07,0.711164,1.0,34.667080,42.819684,36.818704,19.833628,16.766233,113.054805,51.839261,70.251873,37.526235,14.423634,10.443636
std,3.694559e+07,0.453222,0.0,13.845899,39.421731,30.906558,28.788213,19.384930,387.024101,303.629875,183.929622,121.328685,18.034089,11.680559
min,3.000000e+01,0.000000,1.0,16.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,5.794462e+07,0.000000,1.0,25.000000,0.000000,10.000000,0.000000,1.670000,6.000000,3.000000,7.000000,3.000000,2.000000,1.000000
50%,1.038872e+08,1.000000,1.0,31.000000,33.330000,32.830000,6.670000,10.000000,25.000000,11.000000,19.000000,9.000000,6.000000,6.000000
75%,1.162896e+08,1.000000,1.0,41.000000,84.850000,55.560000,26.670000,25.000000,96.000000,37.000000,58.000000,26.000000,20.000000,16.000000
max,1.181887e+08,1.000000,1.0,99.000000,100.000000,100.000000,100.000000,100.000000,120826.000000,120815.000000,8371.000000,5725.000000,60.000000,60.000000


In [5]:
# all_out, 60_out
# Пользователи, у которых были >= 60 исходящих лайков или скипов за время записи.
d = df[df['rate.c.0.0'] >= 60]
pd.concat([d[d.sex == 0]['rate.p.0.0'].describe(), d[d.sex == 0]['rate.p.1.0'].describe(),
           d[d.sex == 1]['rate.p.0.0'].describe(), d[d.sex == 1]['rate.p.1.0'].describe()],axis = 1)

,rate.p.0.0,rate.p.1.0,rate.p.0.0,rate.p.1.0
count,32053.000000,32053.000000,83737.000000,83737.000000
mean,43.243717,46.510469,43.049980,43.155743
std,34.345004,38.586316,36.279788,37.644686
min,0.000000,0.000000,0.000000,0.000000
25%,9.800000,8.330000,8.970000,6.670000
50%,37.310000,40.000000,32.500000,31.670000
75%,75.090000,90.000000,80.820000,83.330000
max,100.000000,100.000000,100.000000,100.000000


In [6]:
pd.concat([d[d.sex == 0]['rate.p.0.0'].quantile([0.33,0.66]), d[d.sex == 0]['rate.p.1.0'].quantile([0.33,0.66]),
           d[d.sex == 1]['rate.p.0.0'].quantile([0.33,0.66]), d[d.sex == 1]['rate.p.1.0'].quantile([0.33,0.66])],axis = 1)

,rate.p.0.0,rate.p.1.0,rate.p.0.0,rate.p.1.0
0.33,16.02,13.33,14.4288,13.33
0.66,61.43,70.00,61.2400,63.33


In [7]:
# women: 20 / 60
# men: 30 / 80

In [8]:
d = df[(df['rate.c.0.0'] >= 60) & (df.age >= 40)]
pd.concat([d[d.sex == 0]['rate.p.0.0'].describe(), d[d.sex == 0]['rate.p.1.0'].describe(),
           d[d.sex == 1]['rate.p.0.0'].describe(), d[d.sex == 1]['rate.p.1.0'].describe()],axis = 1)

,rate.p.0.0,rate.p.1.0,rate.p.0.0,rate.p.1.0
count,8346.000000,8346.000000,20469.000000,20469.000000
mean,27.811716,29.333338,36.066740,36.111798
std,30.600735,34.356283,34.471721,35.792978
min,0.000000,0.000000,0.000000,0.000000
25%,3.450000,1.670000,5.580000,5.000000
50%,13.935000,11.670000,22.970000,21.670000
75%,45.950000,50.000000,64.350000,66.670000
max,100.000000,100.000000,100.000000,100.000000


In [13]:
d = df[(df['rate.c.0.2'] >= 60) & (df['rate.c.0.0'] >= 60)]
pd.concat([d[d.sex == 0]['rate.p.0.0'].describe(), d[d.sex == 0]['rate.p.1.0'].describe(),
           d[d.sex == 1]['rate.p.0.0'].describe(), d[d.sex == 1]['rate.p.1.0'].describe(),
           d[d.sex == 0]['rate.p.0.1'].describe(), d[d.sex == 0]['rate.p.1.1'].describe(),
           d[d.sex == 1]['rate.p.0.1'].describe(), d[d.sex == 1]['rate.p.1.1'].describe()],axis = 1)

,rate.p.0.0,rate.p.1.0,rate.p.0.0,rate.p.1.0,rate.p.0.1,rate.p.1.1,rate.p.0.1,rate.p.1.1
count,28237.000000,28237.000000,10725.000000,10725.000000,28237.000000,28237.000000,10725.000000,10725.000000
mean,46.307054,49.896680,56.814902,57.616780,44.765019,44.497484,42.722316,43.185833
std,33.595507,37.956494,34.987213,36.832197,14.608632,15.752672,29.668628,29.902943
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.430000,11.670000,22.290000,20.000000,34.090000,33.330000,17.930000,18.330000
50%,43.530000,46.670000,59.930000,61.670000,45.310000,45.000000,34.290000,35.000000
75%,77.780000,93.330000,93.160000,96.670000,55.680000,56.670000,67.160000,68.330000
max,100.000000,100.000000,100.000000,100.000000,92.900000,98.330000,100.000000,100.000000


In [18]:
d = df[(df['rate.c.0.2'] >= 60) & (df['rate.c.0.0'] >= 60) & (df['rate.p.0.0'] >= 40)]
pd.concat([d[d.sex == 0]['rate.p.0.0'].describe(), d[d.sex == 0]['rate.p.1.0'].describe(),
           d[d.sex == 1]['rate.p.0.0'].describe(), d[d.sex == 1]['rate.p.1.0'].describe(),
           d[d.sex == 0]['rate.p.0.1'].describe(), d[d.sex == 0]['rate.p.1.1'].describe(),
           d[d.sex == 1]['rate.p.0.1'].describe(), d[d.sex == 1]['rate.p.1.1'].describe()],axis = 1)

,rate.p.0.0,rate.p.1.0,rate.p.0.0,rate.p.1.0,rate.p.0.1,rate.p.1.1,rate.p.0.1,rate.p.1.1
count,14813.000000,14813.00000,6682.000000,6682.000000,14813.000000,14813.000000,6682.000000,6682.000000
mean,74.584114,80.31093,80.796022,81.534431,46.776915,46.482939,50.163045,50.643261
std,18.772025,22.78733,18.945191,22.336935,14.951223,16.116043,28.964601,29.168181
min,40.000000,0.00000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,58.330000,63.33000,65.490000,66.670000,36.010000,35.000000,24.705000,25.000000
50%,75.930000,90.00000,87.780000,91.670000,47.930000,46.670000,46.865000,46.670000
75%,92.650000,100.00000,97.830000,100.000000,58.100000,58.330000,76.105000,76.670000
max,100.000000,100.00000,100.000000,100.000000,92.900000,98.330000,100.000000,100.000000


In [19]:
d = df[(df['rate.c.0.2'] >= 60) & (df['rate.c.0.0'] >= 60) & (df['rate.p.0.0'] >= 75)]
pd.concat([d[d.sex == 0]['rate.p.0.0'].describe(), d[d.sex == 0]['rate.p.1.0'].describe(),
           d[d.sex == 1]['rate.p.0.0'].describe(), d[d.sex == 1]['rate.p.1.0'].describe(),
           d[d.sex == 0]['rate.p.0.1'].describe(), d[d.sex == 0]['rate.p.1.1'].describe(),
           d[d.sex == 1]['rate.p.0.1'].describe(), d[d.sex == 1]['rate.p.1.1'].describe()],axis = 1)

,rate.p.0.0,rate.p.1.0,rate.p.0.0,rate.p.1.0,rate.p.0.1,rate.p.1.1,rate.p.0.1,rate.p.1.1
count,7603.000000,7603.000000,4414.000000,4414.000000,7603.000000,7603.000000,4414.000000,4414.000000
mean,90.729467,95.994722,92.970763,93.561289,46.381593,46.242271,56.177664,56.628527
std,7.612777,9.030441,7.229650,11.527444,15.284678,16.431549,27.877419,28.086166
min,75.000000,0.000000,75.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,84.410000,96.670000,88.060000,91.670000,35.455000,35.000000,32.622500,33.330000
50%,92.360000,100.000000,95.895000,98.330000,47.170000,46.670000,56.960000,56.670000
75%,97.540000,100.000000,99.110000,100.000000,57.585000,58.330000,80.995000,81.670000
max,100.000000,100.000000,100.000000,100.000000,92.530000,98.330000,100.000000,100.000000
